## Datenaufbereitung der Daten über die Sonneneruptionen
Im vorliegenden Notizbuch werden die Daten über die Sonneneruptionen zwischen 1975 und 2016 von NOAA geladen und aufbereitet.<br>

Datenquelle:https://www.ngdc.noaa.gov/stp/space-weather/solar-data/solar-features/solar-flares/x-rays/goes/xrs/ <br>
Dokumentation der Spalten: https://www.ngdc.noaa.gov/stp/space-weather/solar-data/solar-features/solar-flares/x-rays/goes/xrs/documentation/miscellaneous/software/xraydatareports.pro

## Inhaltsverzeichnis

1. [Importe](#1.-Importe)<br>
2. [Download der Daten](#2.-Download-der-Daten)<br>
3. [Erstellung der Spalte 'date' und Umrechnung der Stärken-Klassen](#3.-Erstellung-der-Spalte-'date'-und-Umrechnung-der-Stärken-Klassen)<br>
    3.1 [Erstellung der Spalte 'date' und Umrechnung der Stärken-Klassen](#3.1-Erstellung-der-Spalte-'date'-und-Umrechnung-der-Stärken-Klassen)<br>
    3.2 [Reduktion auf einen Wert pro Tag](#3.2-Reduktion-auf-einen-Wert-pro-Tag)<br>
    3.3 [Aufbereitung für Analyse und Berechnungen in R](#3.3-Aufbereitung-für-Analyse-und-Berechnungen-in-R)<br>
    3.4 [Aufbereitung für Survival Function und Zipf-Plots](#3.4-Aufbereitung-für-Survival-Function-und-Zipf-Plots)<br>
    3.4.1 [Erstellung CSV-Datei mit Daten aus Survival Function](#3.4.1-Erstellung-CSV-Datei-mit-Daten-aus-Survival-Function)<br>
    3.4.2 [Erstellung CSV-Datei mit aggregierten Daten (1x) für Zipf-Plot](#3.4.2-Erstellung-CSV-Datei-mit-aggregierten-Daten-(1x)-für-Zipf-Plot)<br>
    3.4.3 [Erstellung CSV-Datei mit aggregierten Daten (2x) für Zipf-Plot](#3.4.3-Erstellung-CSV-Datei-mit-aggregierten-Daten-(2x)-für-Zipf-Plot)<br>
    3.4.4 [Erstellung CSV-Datei mit aggregierten Daten (3x) für Zipf-Plot](#3.4.4-Erstellung-CSV-Datei-mit-aggregierten-Daten-(3x)-für-Zipf-Plot)<br>
4. [Beispiele Umrechnung von Klasse zu W/m2](#4.-Beispiele-Umrechnung-von-Klasse-zu-W/m2)

## 1. Importe

In [1]:
import numpy as np
import pandas as pd
import requests

## 2. Download der Daten
Die Daten werden direkt von der Webseite von NOAA geholt und im Unterordner 'data_solar_flare_raw' pro Jahr untergespeichert. Anschliessend werden die Daten von allen Jahren zusammenfasst und in der CSV-Datei 'solar_flare_new.csv' gespeichert.

In [2]:
df_sf_concat = pd.DataFrame(columns=['Data code', 'Station code', 'Year', 'Month', 'Day', 'Astriks', 
                         'Start time', 'End time', 'Max time', 'N or S', 'Latitude N/S', 
                         'E or W', 'Central mer. distance', 'SXI', 'X-ray Class', 'X-ray intensity',
                         'Station Name', 'Integrated flux', 'Sunspot region nr', 'Year CMP',
                         'Month CMP', 'Day CMP', 'Total region area', 'Total intensity'])
url = 'https://www.ngdc.noaa.gov/stp/space-weather/solar-data/solar-features/solar-flares/x-rays/goes/xrs/goes-xrs-report_YEAR.txt'

for year in range(1975,2017): #2017
    txt = requests.get(url.replace("YEAR", str(year)))
    with open('data_files/data_solar_flare_raw/solar_flare_raw_' + str(year) +'.csv', 'w') as f:
        f.write(txt.text)
        
    df_solar_flare = pd.read_csv('data_files/data_solar_flare_raw/solar_flare_raw_' + str(year) +'.csv', header=None, names=["text"])

    df_solar_flare['Data code'] = df_solar_flare.text.str[:2]
    df_solar_flare['Station code'] = df_solar_flare.text.str[2:5]
    df_solar_flare['Year'] = df_solar_flare.text.str[5:7]
    df_solar_flare['Year_long'] = year
    df_solar_flare['Month'] = df_solar_flare.text.str[7:9]
    df_solar_flare['Day'] = df_solar_flare.text.str[9:11]
    df_solar_flare['Astriks'] = df_solar_flare.text.str[11:13]
    df_solar_flare['Start time'] = df_solar_flare.text.str[13:17]
    df_solar_flare['End time'] = df_solar_flare.text.str[18:22]
    df_solar_flare['Max time'] = df_solar_flare.text.str[23:27]
    df_solar_flare['N or S'] = df_solar_flare.text.str[28:29]
    df_solar_flare['Latitude N/S'] = df_solar_flare.text.str[29:31]
    df_solar_flare['E or W'] = df_solar_flare.text.str[31:32]
    df_solar_flare['Central mer. distance'] = df_solar_flare.text.str[32:34]
    df_solar_flare['SXI'] = df_solar_flare.text.str[34:37]
    df_solar_flare['X-ray Class'] = df_solar_flare.text.str[59:60]
    df_solar_flare['X-ray intensity'] = df_solar_flare.text.str[60:63]
    df_solar_flare['Station Name'] = df_solar_flare.text.str[67:71]
    df_solar_flare['Integrated flux'] = df_solar_flare.text.str[72:80]
    df_solar_flare['Sunspot region nr'] = df_solar_flare.text.str[80:85]
    df_solar_flare['Year CMP'] = df_solar_flare.text.str[86:88]
    df_solar_flare['Month CMP'] = df_solar_flare.text.str[88:90]
    df_solar_flare['Day CMP'] = df_solar_flare.text.str[90:94]
    df_solar_flare['Total region area'] = df_solar_flare.text.str[95:102]
    df_solar_flare['Total intensity'] = df_solar_flare.text.str[103:110]
    # drop the first column with the raw string
    df_solar_flare.drop(columns=['text'], inplace=True)

    df_sf_concat = pd.concat([df_sf_concat, df_solar_flare], axis=0)
    
df_sf_concat.to_csv('data_files/solar_flare_new.csv', sep=';', index=False)

## 3. Aufbereitung der Daten

### 3.1 Erstellung der Spalte 'date' und Umrechnung der Stärken-Klassen
Aus allen Informationen werden nur die Intensität und das jeweilige Datum der Sonneneruption benötigt. Um das Datum zu erhalten werden die Spalten 'Year_long', 'Month' und 'Day' zu einer Datums-Spalte zusammengefasst. Damit aus der Klassifizierung der Sonneneruptionen nummerische Werte entstehen, werden sie in W/m2 umgerechnet. Beispiele für die Umrechnung sind unter [4. Beispiele Umrechnung von Klasse zu W/m2](#4.-Beispiele-Umrechnung-von-Klasse-zu-W/m2) ersichtlich.

In [3]:
# Daten aufbereiten, damit sie für die Berechnungen benutzt werden können
df_sf = pd.read_csv('data_files/solar_flare_new.csv', sep=';', low_memory=False,
                    usecols=['Year_long', 'Month', 'Day', 'X-ray Class', 'X-ray intensity'])

date = pd.to_datetime(dict(year=df_sf.Year_long, month=df_sf.Month, day=df_sf.Day))
df_sf.insert(0, "date", date, True)
df_sf.drop(columns=['Year_long', 'Month', 'Day'], inplace=True)

df_sf['X-ray intensity'].str.strip()
df_sf['X-ray intensity'] = df_sf['X-ray intensity'].apply(pd.to_numeric, errors='ignore')
df_sf = df_sf.dropna()
df_sf['X-ray intensity'] = np.where(df_sf['X-ray intensity'].astype(str).str.isnumeric(), df_sf['X-ray intensity'], 0)

df_sf['X-ray intensity'] = df_sf['X-ray intensity'] / 10

df_sf['XRay_class_nr1'] = np.where(df_sf['X-ray Class']== 'A', df_sf['X-ray intensity'] * 10 ** -8, 
                                 np.where(df_sf['X-ray Class']== 'B', df_sf['X-ray intensity'] * 10 ** -7, 
                                     np.where(df_sf['X-ray Class']== 'C', df_sf['X-ray intensity'] * 10 ** -6, 
                                         np.where(df_sf['X-ray Class']== 'M', df_sf['X-ray intensity'] * 10 ** -5, 
                                                 np.where(df_sf['X-ray Class']== 'X', df_sf['X-ray intensity'] * 10 ** -4, 0
)))))

# Entferne alle Datensätze mit "X-ray intensity" = 0
df_sf = df_sf[(df_sf.XRay_class_nr1 > 0)]

df_test = df_sf

df_sf.set_index('date', inplace=True)

df_sf.to_csv('data_files/solar_flare_prepared.csv', sep=';')

### 3.2 Reduktion auf einen Wert pro Tag
Ziel: eine CSV-Datei mit dem richtigen Format für die Anwendung von pyextremes erstellen<br>
Step 1: Aufbereitete Daten in das Dataframe "ds_sf" importieren (nur Spalten 'date' und 'XRay_class_nr1')<br>
Step 2: Spalte 'date' (Datum) als Index setzen<br>

Zwischen-Info: Da das Dataframe dt_temp aktuell nicht für jeden Tag Werte hat und für manche Tage mehrere Werte, muss das Dataframe umgewandelt werden, sodass für jeden Tag exakt ein Wert existiert.<br>

Step 3: Erstelle ein neues Dataframe mit einem Datums-Index von Anfang bis Ende des Original-Dataframes<br>
Step 4: Füge die Spalte "Sunspots" hinzu und fülle sie mit 0 (-> alle Tage ohne Wert erhalten neu 0 als Wert)<br>
Step 5: Schleife durch die Daten im Original-Dataframe und fülle das neue Dataframe jeweils mit dem höchstem Wert pro Tag<br>
Step 6: Speichere das neue Dataframe als "df_extreme.csv ab"<br>

In [4]:
# Aufbereitete Daten in das Dataframe "ds_sf" importieren (nur Spalten 'date' und 'XRay_class_nr1')
df_temp = pd.read_csv('data_files/solar_flare_prepared.csv', sep=';' ,usecols=['date', 'XRay_class_nr1'])

# Spalte 'date' (Datum) als Index setzen
df_temp.set_index('date', inplace=True)

# Da das Dataframe dt_temp aktuell nicht für jeden Tag Werte hat und für manche Tage mehrere Werte,
# muss das Dataframe umgewandelt werden, sodass für jeden Tag ein Wert und pro Tag nur ein Wert existiert.

# Erstelle ein neues Dataframe mit einem Datums-Index von Anfang bis Ende des Original-Dataframes
date_range = pd.date_range(start=df_temp.index.min(), end=df_temp.index.max(), freq='D')
df_new = pd.DataFrame(index=date_range)

# Füge die Spalte "Sunspots" hinzu und fülle sie mit 0 (-> alle Tage ohne Wert erhalten neu 0 als Wert)
df_new['XRay_class_nr1'] = 0

# Schleife durch die Daten im Original-Dataframe und fülle das neue Dataframe entsprechend
for index, row in df_temp.iterrows():
    # Wenn ein Eintrag für diesen Tag im Original-Dataframe existiert, füge den höchsten Wert hinzu
    if index in df_new.index:
        max_sf = df_temp.loc[index]['XRay_class_nr1'].max()
        df_new.at[index, 'XRay_class_nr1'] = max_sf

# Speichere das neue Dataframe als "df_extreme.csv ab"
df_new.to_csv('data_files/df_extreme.csv')

### 3.3 Aufbereitung für Analyse und Berechnungen in R
Für die Datenanalyse und die Berechnungen in R wird eine CSV-Datei nur mit der Spalte 'XRay_class_nr1' (umgerechnete Intensitäten) und ohne Header erstellt.

In [5]:
# Aufbereitetes Dataframe importieren
df_R = pd.read_csv('data_files/df_extreme.csv', sep=',',usecols=['XRay_class_nr1'])

# in Datei df_extreme_neu.csv speichern
df_R.to_csv('data_files/R_df_solarflare.csv', index=False, header=False)

### 3.4 Aufbereitung für Survival Function und Zipf-Plots

#### 3.4.1 Erstellung CSV-Datei mit Daten aus Survival Function

In [6]:
# Aufbereitetes Dataframe importieren
df_sf = pd.read_csv('data_files/df_extreme.csv', sep=',')

# Erstelle neues DF für die Survival Function
df_survival = df_sf[['XRay_class_nr1']].copy()

# Sortiere das DataFrame nach den Anzahl Sonnenflecken und resete den Index
df_survival.sort_values("XRay_class_nr1", inplace=True)
df_survival.reset_index(drop=True, inplace=True)

# Die Funktion gibt die Anzahl Werte der Spalte zurück, welche grösser oder gleich sind 
# wie der Wert der aktuellen Zeile
def calc(x):
    counter = 0
    for i in df_survival.XRay_class_nr1:
        if x >= i:
            counter = counter +1
    return counter-1

# Erstelle die neue Spalte "Count" mit der Anzahl grösserer oder gleicher Werte
df_survival['count'] = df_survival['XRay_class_nr1'].apply(calc)

# Speichere das DF als CSV, da die Berechnung der Spalte "Count" sehr lange dauert
df_survival.to_csv('data_files/survival_sf.csv', sep=';', index=False)

#### 3.4.2 Erstellung CSV-Datei mit aggregierten Daten (1x) für Zipf-Plot

In [7]:
df_survival = pd.read_csv('data_files/survival_sf.csv', sep=';')

df_temp = df_survival[(df_survival['XRay_class_nr1'] > 0)]['XRay_class_nr1']
df_zipf_1 = pd.DataFrame(columns=['XRay_class_nr1'])

min = 0
max = df_temp.count() -1

loop_max = int(max/2)

print('loop_max: ',loop_max)
print('min: ',min)    
print('max: ',max)    

for x in range(0, loop_max):
    df_zipf_1.loc[x, 'XRay_class_nr1'] = df_temp.iloc[max-x] + df_temp.iloc[min+x]
    
df_zipf_1.to_csv('data_files/R_df_solarflare_zipf_1.csv', index=False, header=False)

loop_max:  5802
min:  0
max:  11604


#### 3.4.3 Erstellung CSV-Datei mit aggregierten Daten (2x) für Zipf-Plot

In [8]:
df_zipf_2 = pd.DataFrame(columns=['XRay_class_nr1'])

min = 0
max = df_zipf_1['XRay_class_nr1'].count() -1

loop_max = int(max/2)

print('loop_max: ',loop_max)
print('min: ',min)    
print('max: ',max)    

for x in range(0, loop_max):
    df_zipf_2.loc[x, 'XRay_class_nr1'] = df_zipf_1.iloc[max-x]['XRay_class_nr1'] + df_zipf_1.iloc[min+x]['XRay_class_nr1']
    
df_zipf_2.to_csv('data_files/R_df_solarflare_zipf_2.csv', index=False, header=False)

loop_max:  2900
min:  0
max:  5801


#### 3.4.4 Erstellung CSV-Datei mit aggregierten Daten (3x) für Zipf-Plot

In [9]:
df_zipf_3 = pd.DataFrame(columns=['XRay_class_nr1'])

min = 0
max = df_zipf_2['XRay_class_nr1'].count() -1

loop_max = int(max/2)

print('loop_max: ',loop_max)
print('min: ',min)    
print('max: ',max)    

for x in range(0, loop_max):
    df_zipf_3.loc[x, 'XRay_class_nr1'] = df_zipf_2.iloc[max-x]['XRay_class_nr1'] + df_zipf_2.iloc[min+x]['XRay_class_nr1']
    
df_zipf_3.to_csv('data_files/R_df_solarflare_zipf_3.csv', index=False, header=False)

loop_max:  1449
min:  0
max:  2899


## 4. Beispiele Umrechnung von Klasse zu W/m2
Nachfolgend sind einige Beispiel-Rechnungen aufgeführt, wie die Klassifizierung der Sonneneruptionen in nummerische Werte umgewandelt wird.

In [10]:
a1 = 10 ** -8
print('A1:', a1)

b1 = 10 ** -7
print('B1:', b1)

c1 = 10 ** -6
print('C1:', c1)

m1 = 10 ** -5
print('M1:', m1)

x1 = 10 ** -4
print('X1:', x1)

a22 = 2.2 * 10 ** -8
print('A2.2:', a22)

b22 = 2.2 * 10 ** -7
print('B2.2:', b22)

c22 = 2.2 * 10 ** -6
print('C2.2:', c22)

m22 = 2.2 * 10 ** -5
print('M2.2:', m22)

x22 = 2.2 * 10 ** -4
print('X2.2:', x22)

a99 = 9.9 * 10 ** -8
print('A9.9:', a99)

b99 = 9.9 * 10 ** -7
print('B9.9:', b99)

c99 = 9.9 * 10 ** -6
print('C9.9:', c99)

m99 = 9.9 * 10 ** -5
print('M9.9:', m99)

x99 = 9.9 * 10 ** -4
print('X9.9:', x99)

# carrington like event
x450 = 45 * 10 ** -4
print('X45:', x450)

A1: 1e-08
B1: 1e-07
C1: 1e-06
M1: 1e-05
X1: 0.0001
A2.2: 2.2000000000000002e-08
B2.2: 2.2e-07
C2.2: 2.2e-06
M2.2: 2.2000000000000003e-05
X2.2: 0.00022000000000000003
A9.9: 9.9e-08
B9.9: 9.9e-07
C9.9: 9.9e-06
M9.9: 9.900000000000001e-05
X9.9: 0.00099
X45: 0.0045000000000000005
